In [3]:
# Import modules
import numpy as np
import librosa.display, os
import matplotlib.pyplot as plt
import keras.utils as image

import gc
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, classification_report, accuracy_score, roc_curve
import numpy as np
from keras.layers import GlobalAveragePooling2D
from tensorflow.keras.applications.mobilenet import MobileNet

import tensorflow as tf
from tensorflow.keras import layers, models

# Import modules
import tensorflow as tf
import tensorflow.keras as K


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
import keras

# import keras.utils as image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

2025-03-08 15:40:17.859354: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-08 15:40:17.870213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741423217.881307  362110 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741423217.884767  362110 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-08 15:40:17.897842: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [ ]:
N_MFCC = 128  

In [ ]:
X = np.load('../SavedFeatures/X_mfcc.npy')
y = np.load('../SavedFeatures/y_mfcc.npy')


X_train, X_val, y_train, y_val= train_test_split(X, y, test_size = 0.2, random_state = 42)

y_train = to_categorical(y_train, num_classes=2)
y_val = to_categorical(y_val, num_classes=2)

del X, y  
gc.collect()

In [ ]:
def model_resnet(input_shape, num_classes):
    base_model = ResNet50(include_top=False, weights=None, input_shape=input_shape)
    x = Flatten()(base_model.output)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.4)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.3)(x)
    
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    return Model(inputs=base_model.input, outputs=x)

In [21]:
tf.keras.backend.clear_session()

In [23]:
mrsnt = model_resnet(input_shape=(N_MFCC, 109, 1), num_classes=2)
mrsnt.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])


hist = mrsnt.fit(X_train, y_train, batch_size=16, epochs=10, validation_data=(X_val, y_val))

Epoch 1/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 53s 53ms/step - accuracy: 0.5808 - loss: 0.9870 - val_accuracy: 0.9128 - val_loss: 0.4681
Epoch 2/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.8671 - loss: 0.4494 - val_accuracy: 0.9305 - val_loss: 0.3494
Epoch 3/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9343 - loss: 0.3542 - val_accuracy: 0.9690 - val_loss: 0.2541
Epoch 4/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9556 - loss: 0.2564 - val_accuracy: 0.8850 - val_loss: 0.3150
Epoch 5/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 24s 38ms/step - accuracy: 0.9532 - loss: 0.1773 - val_accuracy: 0.8053 - val_loss: 0.6057
Epoch 6/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 24s 37ms/step - accuracy: 0.9575 - loss: 0.1443 - val_accuracy: 0.9537 - val_loss: 0.1936
Epoch 7/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 21s 33ms/step - accuracy: 0.9691 - loss: 0.1083 - val_accuracy: 0.7546 - val_loss: 0.8004
Epoch 8/10
637/637 ━━━━━━━━━━━━━━━━━━━━ 20s 32ms/step - accuracy: 0.9691 - loss: 0.1126 - 

In [ ]:
test_loss, test_accuracy = mrsnt.evaluate(X_val, y_val, verbose=0)
y_pred = mrsnt.predict(X_val)
y_pred_classes = y_pred.argmax(axis=1) 
y_true_classes = y_val.argmax(axis=1) 

f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')

print("===================ResNet50 - MFCC===================")
print(f"TAccuracy: {test_accuracy:.5f}")
print(f"F1-Score: {f1:.5f}")
eers = []

# Tính EER
for i in range(y_pred.shape[1]):  
    
    y_true_binary = y_val[:, i]
    y_pred_prob = y_pred[:, i]

    
    fpr, tpr, thresholds = roc_curve(y_true_binary, y_pred_prob)
    fnr = 1 - tpr
    
    eer_threshold = thresholds[np.nanargmin(np.abs(fpr - fnr))]
    eer = fpr[np.nanargmin(np.abs(fpr - fnr))]
    eers.append(eer)
    print(f"Class {i}: EER = {eer:.5f} at threshold {eer_threshold:.5f}")


mean_eer = np.mean(eers)
print(f"EER: {mean_eer:.5f}")


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 45ms/step
===================ResNet50 - MFCC===================
TAccuracy: 0.87711
F1-Score: 0.87177
Class 0: EER = 0.03914 at threshold 0.99816
Class 1: EER = 0.03869 at threshold 0.00190
EER: 0.03891


In [25]:
# Save Model as weight with H5 format
mrsnt.save_weights("../SavedModels/MFCC_mrsnet.weights.h5")

In [ ]:
del mrsnt, X_train, X_val, y_train, y_val  
gc.collect()

94512